In [2]:
import QuantLib as ql
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.stats import norm
import math
import sys
np.set_printoptions(threshold=sys.maxsize)

C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\1367023215.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
file_path = 'Project 3 Data.xlsx'
ETFs = ['XLU', 'XLB', 'XLRE', 'XLE', 'XLP', 'XLI', 'XLC', 'XLF', 'XLY', 'XLV', 'XLK']
ETF_data = {}

for i in ETFs:
    data = pd.read_excel(file_path, sheet_name=i, skiprows=4)
    data['Dates'] = pd.to_datetime(data['Dates'])
    data_filtered = data[['Dates','PX_LAST','1M_CALL_IMP_VOL_50DELTA_DFLT']]
    ETF_data[i] =data_filtered
    
SPX = pd.read_excel(file_path, sheet_name='SPX', skiprows=4)
SPX['Dates'] = pd.to_datetime(SPX['Dates'])
SPX = SPX[['Dates','PX_LAST','1M_CALL_IMP_VOL_50DELTA_DFLT']]

In [4]:
risk_free_rates = pd.read_excel('swap_curve.xlsx')
risk_free_rates['Dates'] = pd.to_datetime(risk_free_rates['Dates'])

In [5]:
# Black-Scholes Formulas for European Call and Put Options
def black_scholes_call(spot_price, strike, risk_free_rate, volatility, time_to_maturity, div_yield):
    d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)
    call_price = (spot_price * np.exp(-div_yield * time_to_maturity) * norm.cdf(d1)) - (strike * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2))
    return call_price

def black_scholes_put(spot_price, strike, risk_free_rate, volatility, time_to_maturity, div_yield):
    d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)
    put_price = (strike * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2)) - (spot_price * np.exp(-div_yield * time_to_maturity) * norm.cdf(-d1))
    return put_price

def black_scholes_call_delta(spot_price, strike, risk_free_rate, volatility, time_to_maturity, div_yield):
    d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    call_delta = np.exp(-div_yield * time_to_maturity) * norm.cdf(d1)
    return call_delta

def black_scholes_put_delta(spot_price, strike, risk_free_rate, volatility, time_to_maturity, div_yield):
    d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    put_delta = np.exp(-div_yield * time_to_maturity) * (norm.cdf(d1) - 1)
    return put_delta


In [6]:
# Read initial Last_Price and volatility from the data for each ETF and SPX on 10/18/2021
last_price = {}
volatility = {}
for i in ETFs:
    last_price[i] = ETF_data[i].loc[ETF_data[i]['Dates'] == '2021-10-18']['PX_LAST'].values[0]
    volatility[i] = ETF_data[i].loc[ETF_data[i]['Dates'] == '2021-10-18']['1M_CALL_IMP_VOL_50DELTA_DFLT'].values[0] / 100

last_price['SPX'] = SPX.loc[SPX['Dates'] == '2021-10-18']['PX_LAST'].values[0]
volatility['SPX'] = SPX.loc[SPX['Dates'] == '2021-10-18']['1M_CALL_IMP_VOL_50DELTA_DFLT'].values[0] /100

# initial risk-free rate
risk_free_rate = risk_free_rates.loc[risk_free_rates['Dates'] == '2021-10-18']['Rates'].values[0] / 100

In [7]:
# Create a dataframe to store the dates, last prices for SPX and each ETF
df_price = pd.DataFrame()
df_price['Dates'] = SPX['Dates']
df_price['SPX'] = SPX['PX_LAST']
for i in ETFs:
    df_price[i] = ETF_data[i]['PX_LAST']

# Create a dataframe to store the dates, volatilities for SPX and each ETF
df_vol = pd.DataFrame()
df_vol['Dates'] = SPX['Dates']
df_vol['SPX'] = SPX['1M_CALL_IMP_VOL_50DELTA_DFLT']
for i in ETFs:
    df_vol[i] = ETF_data[i]['1M_CALL_IMP_VOL_50DELTA_DFLT']

In [8]:
# initial conditions
strike_price_SPX = df_price.loc[df_price['Dates'] == '2021-10-18']['SPX'].values[0]

df_price["call_spx"] = np.NaN
df_price["put_spx"] = np.NaN
df_price["call_delta_spx"] = np.NaN
df_price["put_delta_spx"] = np.NaN

for i in range(0, len(df_price)):
    S = df_price.at[i, 'SPX']
    r = risk_free_rates.loc[risk_free_rates['Dates'] == df_price.at[i, 'Dates']]['Rates'].values[0] / 100
    sigma = df_vol.at[i, 'SPX'] / 100
    T = (df_price.at[len(df_price)-1, 'Dates'] - df_price.at[i, 'Dates']).days / 365

    df_price.at[i, 'call_spx'] = black_scholes_call(S, strike_price_SPX, r, sigma, T, 0)
    df_price.at[i, 'put_spx'] = black_scholes_put(S, strike_price_SPX, r, sigma, T, 0)
    df_price.at[i, 'call_delta_spx'] = black_scholes_call_delta(S, strike_price_SPX, r, sigma, T, 0)
    df_price.at[i, 'put_delta_spx'] = black_scholes_put_delta(S, strike_price_SPX, r, sigma, T, 0)

C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / 

In [9]:
df_price['delta'] = df_price['call_delta_spx'] + df_price['put_delta_spx']
df_price['straddle'] = df_price['call_spx'] + df_price['put_spx']

In [10]:
# Create a dataframe to store the dates, call and put prices for each ETF
df = pd.DataFrame()
df['Dates'] = df_price['Dates']

for etf in ETFs:
    df["call_" + etf] = np.NaN
    df["put_" + etf] = np.NaN
    df["Straddle_" + etf] = np.NaN
    df["call delta_" + etf] = np.NaN
    df["put delta_" + etf] = np.NaN
    strike_price_etf = df_price.loc[df_price['Dates'] == '2021-10-18'][etf].values[0]

    for i in range(0, len(df_price)):
        S = df_price.at[i, etf]
        r = risk_free_rates.loc[risk_free_rates['Dates'] == df_price.at[i, 'Dates']]['Rates'].values[0] / 100
        sigma = df_vol.at[i, etf] / 100
        T = (df_price.at[len(df_price)-1, 'Dates'] - df_price.at[i, 'Dates']).days / 365

        df.at[i, "call_" + etf] = black_scholes_call(S, strike_price_etf, r, sigma, T, 0)
        df.at[i, "put_" + etf] = black_scholes_put(S, strike_price_etf, r, sigma, T, 0)
        df.at[i, "straddle_" + etf] = df.at[i, "call_" + etf] + df.at[i, "put_" + etf]
        df.at[i, "call delta_" + etf] = black_scholes_call_delta(S, strike_price_etf, r, sigma, T, 0)
        df.at[i, "put delta_" + etf] = black_scholes_put_delta(S, strike_price_etf, r, sigma, T, 0)
        
call_etf = df.filter(like='call')
put_etf = df.filter(like='put')
# sombine the calls together into one column
df['call_etf'] = call_etf.sum(axis=1)
df['put_etf'] = put_etf.sum(axis=1)

C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / strike) + (risk_free_rate - div_yield + 0.5 * volatility ** 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2887753833.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(spot_price / 

In [11]:
df_price['call_etf'] = df['call_etf']
df_price['put_etf'] = df['put_etf']

In [12]:
df_price

,Dates,SPX,XLU,XLB,XLRE,XLE,XLP,XLI,XLC,XLF,...,XLV,XLK,call_spx,put_spx,call_delta_spx,put_delta_spx,delta,straddle,call_etf,put_etf
0,2021-10-18,4486.46,64.5431,83.6663,45.8866,56.6366,69.9110,102.5731,80.7742,39.3311,...,126.6138,156.9336,92.963098,88.672162,0.517640,-0.482360,0.035280,181.635260,32.767702,20.794160
1,2021-10-19,4519.63,65.3567,84.1041,46.0948,57.2783,69.9408,103.1808,81.5526,39.7193,...,128.2775,158.4907,106.815286,69.546105,0.577405,-0.422595,0.154810,176.361392,36.276695,17.759659
2,2021-10-20,4536.19,66.3886,84.7806,46.7988,57.7423,70.3480,103.8283,81.7122,40.0377,...,130.1104,158.0515,114.030895,60.312473,0.609175,-0.390825,0.218350,174.343369,39.556106,15.401200
3,2021-10-21,4549.78,66.3886,84.5617,46.9475,56.6761,70.2586,104.0376,81.9517,39.8686,...,130.6384,158.6604,119.055351,51.351580,0.639440,-0.360560,0.278881,170.406931,40.502881,13.271676
4,2021-10-22,4544.90,66.7259,84.4025,47.2846,57.2092,70.8049,104.4062,80.2153,40.4259,...,131.1664,158.1114,117.917632,55.178985,0.627513,-0.372487,0.255025,173.096617,41.636233,14.350242
5,2021-10-25,4566.48,66.4382,85.2483,47.4234,58.0484,70.7552,104.6453,80.5646,40.3761,...,131.3258,158.6304,128.030260,44.017945,0.672201,-0.327799,0.344402,172.048205,45.860761,11.167971
6,2021-10-26,4574.79,66.7954,85.4870,47.6416,58.3840,71.0035,103.9977,79.8461,40.4060,...,131.9733,159.0796,136.803012,44.494684,0.680396,-0.319604,0.360793,181.297696,48.074578,12.196462
7,2021-10-27,4551.68,66.3489,84.2632,47.2946,56.7057,70.5864,102.7823,79.6265,39.7392,...,130.9273,158.9398,126.832610,57.467746,0.632855,-0.367145,0.265709,184.300356,43.993947,15.039047
8,2021-10-28,4596.42,66.7855,85.0990,47.9886,57.1105,70.8347,104.0674,79.8860,40.2268,...,132.0231,160.5368,151.039476,37.035496,0.718784,-0.281216,0.437567,188.074971,50.652343,11.749731
9,2021-10-29,4605.38,66.3886,84.6911,47.4135,56.7354,70.7651,104.0973,80.1355,40.0676,...,133.3081,161.2255,155.083245,31.991303,0.740756,-0.259244,0.481513,187.074548,49.630766,9.910622


In [13]:
df_pnl = pd.DataFrame()
df_pnl['Dates'] = df_price['Dates']

df_pnl['pnl_short_call'] = df_price['call_spx'].diff()
df_pnl['pnl_short_put'] = df_price['put_spx'].diff()
df_pnl['SPX Straddle PnL'] = (df_pnl['pnl_short_call'] + df_pnl['pnl_short_put']) * -1000
df_pnl[['Dates', 'SPX Straddle PnL']]

,Dates,SPX Straddle PnL
0,2021-10-18,NaN
1,2021-10-19,5273.868466
2,2021-10-20,2018.022985
3,2021-10-21,3936.438029
4,2021-10-22,-2689.686503
5,2021-10-25,1048.411882
6,2021-10-26,-9249.490494
7,2021-10-27,-3002.660570
8,2021-10-28,-3774.614964
9,2021-10-29,1000.423649


In [14]:
# Adjust the hedge daily
SPX_prices_diff = df_price['SPX'].diff()  
df_pnl['SPX_Hedge_PnL'] = df_price['delta'].shift()* SPX_prices_diff * -1000 # Shift hedge position for PnL calculation
df_pnl

,Dates,pnl_short_call,pnl_short_put,SPX Straddle PnL,SPX_Hedge_PnL
0,2021-10-18,NaN,NaN,NaN,NaN
1,2021-10-19,13.852188,-19.126057,5273.868466,-1170.233293
2,2021-10-20,7.215609,-9.233632,2018.022985,-2563.652236
3,2021-10-21,5.024455,-8.960893,3936.438029,-2967.374218
4,2021-10-22,-1.137718,3.827405,-2689.686503,1360.938092
5,2021-10-25,10.112628,-11.161040,1048.411882,-5503.449915
6,2021-10-26,8.772751,0.476739,-9249.490494,-2861.981918
7,2021-10-27,-9.970401,12.973062,-3002.660570,8337.919491
8,2021-10-28,24.206866,-20.432251,-3774.614964,-11887.834187
9,2021-10-29,4.043769,-5.044193,1000.423649,-3920.602953


In [15]:
# Weights for each sector ETF based on the Bloomberg screenshot
ETF_weights = {
    'XLK': 27.91488 / 100,
    'XLV': 12.69217 / 100,
    'XLY': 12.618698 / 100,
    'XLF': 11.499232 / 100,
    'XLC': 11.042944 / 100,
    'XLI': 8.127506 / 100,
    'XLP': 5.670948 / 100,
    'XLE': 2.924949 / 100,
    'XLRE': 2.579896 / 100,
    'XLB': 2.527329 / 100,
    'XLU': 2.401839 / 100
}
straddle_price_spx = df_price['straddle'][0]
straddle_prices = {}
for etf in ETFs:
    straddle_prices[etf] = df["straddle_" + etf][0]

# Calculating the number of straddles to long for each sector ETF
num_straddles = {}
shares = 1000
for i in ETFs:
    num_straddles[i] = straddle_price_spx * shares * ETF_weights[i] /straddle_prices[i]

num_straddles

{'XLU': 1401.713017539008,
 'XLB': 876.9210628654349,
 'XLRE': 2086.10381548821,
 'XLE': 1069.9385309791571,
 'XLP': 3708.178945301098,
 'XLI': 2983.379101039445,
 'XLC': 5120.74967690223,
 'XLF': 9255.122345902457,
 'XLY': 2331.243169068156,
 'XLV': 3845.0031866507834,
 'XLK': 6458.098345954561}

In [16]:

strike_price_ETF = {}
for i in ETFs:
    strike_price_ETF[i] = df_price.loc[df_price['Dates'] == '2021-10-18'][i].values[0]

df_pnl2 = pd.DataFrame()
df_pnl2['Dates'] = df_price['Dates']

for etf in ETFs:
    df_pnl2[etf + ' Long Straddle PnL'] = df["straddle_" + etf].diff() * num_straddles[etf]
    # for i in range(0, len(df_price)): 
    #     df_pnl2[etf + ' Long Straddle PnL'] = calculate_long_straddle_pnl(strike_price_ETF[etf], df_price[etf][i], df["Straddle_" + etf][i]) * num_straddles[etf]
df_pnl2['ETF_Straddle_Pnl'] = df_pnl2.filter(like='Straddle').sum(axis=1)
df_pnl['ETF Straddle Pnl'] = df_pnl2['ETF_Straddle_Pnl']

In [17]:
# Adjust the hedge daily

SPX_prices_diff = df_price['SPX'].diff()  
df_pnl['SPX_Hedge_PnL'] = df_price['delta'].shift()* SPX_prices_diff * -1000 # Shift hedge position for PnL calculation
df_pnl

for etf in ETFs:
    df["delta_" + etf] = df["call delta_" + etf] + df["put delta_" + etf]
    df_pnl2[etf + ' Hedge PnL'] = df['delta_' + etf].shift() * (df_price[etf].diff() * num_straddles[etf])
df_pnl2['ETF_Hedge_PnL'] = df_pnl2.filter(like='Hedge').sum(axis=1)
df_pnl["ETF Hedge PnL"] = df_pnl2['ETF_Hedge_PnL']

In [18]:
df_pnl = df_pnl[['Dates','SPX Straddle PnL','SPX_Hedge_PnL','ETF Straddle Pnl',	'ETF Hedge PnL']]
df_pnl['SPX Daily PnL'] = df_pnl['SPX Straddle PnL'] + df_pnl['SPX_Hedge_PnL']
df_pnl['ETF Daily PnL'] = df_pnl['ETF Straddle Pnl'] + df_pnl['ETF Hedge PnL']
df_pnl['Total PnL'] = df_pnl['SPX Daily PnL'] + df_pnl['ETF Daily PnL']
df_pnl = df_pnl.replace(np.nan, 0)
df_pnl['ETF Straddle Pnl'] = round(df_pnl['ETF Straddle Pnl'],0)
df_pnl['ETF Hedge PnL'] = round(df_pnl['ETF Hedge PnL'],0)
df_pnl['ETF Daily PnL'] = round(df_pnl['ETF Daily PnL'],0)
df_pnl = df_pnl[['Dates','SPX Straddle PnL', 'SPX_Hedge_PnL', 'SPX Daily PnL','ETF Straddle Pnl','ETF Hedge PnL','ETF Daily PnL','Total PnL']]
df_pnl

C:\Users\zhong\AppData\Local\Temp\ipykernel_11152\2293487337.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pnl['SPX Daily PnL'] = df_pnl['SPX Straddle PnL'] + df_pnl['SPX_Hedge_PnL']


,Dates,SPX Straddle PnL,SPX_Hedge_PnL,SPX Daily PnL,ETF Straddle Pnl,ETF Hedge PnL,ETF Daily PnL,Total PnL
0,2021-10-18,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
1,2021-10-19,5273.868466,-1170.233293,4103.635173,-417.0,1027.0,610.0,4713.616442
2,2021-10-20,2018.022985,-2563.652236,-545.629251,-4118.0,2554.0,-1564.0,-2109.341841
3,2021-10-21,3936.438029,-2967.374218,969.063811,-3167.0,1072.0,-2095.0,-1126.252169
4,2021-10-22,-2689.686503,1360.938092,-1328.748411,10649.0,-69.0,10580.0,9251.164949
5,2021-10-25,1048.411882,-5503.449915,-4455.038033,-2086.0,2192.0,106.0,-4349.496759
6,2021-10-26,-9249.490494,-2861.981918,-12111.472412,14082.0,2231.0,16313.0,4201.228656
7,2021-10-27,-3002.660570,8337.919491,5335.258921,-4277.0,-7219.0,-11496.0,-6160.787822
8,2021-10-28,-3774.614964,-11887.834187,-15662.449151,-1150.0,8606.0,7456.0,-8206.371211
9,2021-10-29,1000.423649,-3920.602953,-2920.179305,-2435.0,3455.0,1019.0,-1900.827390


In [20]:
# Export the PnL to an excel file
df_pnl.to_excel('PnL.xlsx', index=False)

Compare the implied correlation from the option market on 10/18/2021 and the average realized correlation between 10/18/2021 and 12/17/2021.

In [21]:
implied_vols = {
    'XLU': 14.9137,
    'XLB': 19.3539,
    'XLRE': 15.1403,
    'XLE': 27.1271,
    'XLP': 12.2872,
    'XLI': 14.9199,
    'XLC': 14.9979,
    'XLF': 17.4748,
    'XLY': 15.9178,
    'XLV': 14.6455,
    'XLK': 15.4730,
}
# Convert implied volatilities to variances by squaring them and dividing by 10000 (since they are percentages)
implied_vars = {etf: (vol / 100) ** 2 for etf, vol in implied_vols.items()} 

# Create the components_etf dictionary with weights and variances
components_etf = {etf: (ETF_weights[etf], implied_vars[etf])for etf in ETF_weights.keys()} # (weight, variance)

In [22]:
# Variance of the SP500 index on 10/18/2021
variance_spx = (df_vol.loc[df_vol['Dates'] == '2021-10-18']['SPX'].values[0] / 100) **2

# Calculate the weighted sum of variances and the weighted sum of products of volatilities
weighted_sum_variances = sum(weight**2 * variance for weight, variance in components_etf.values())
weighted_sum_products = sum(weight_i * weight_j * (variance_i**0.5) * (variance_j**0.5)
                            for i, (weight_i, variance_i) in enumerate(components_etf.values())
                            for j, (weight_j, variance_j) in enumerate(components_etf.values()) if i != j)

# Calculate the implied correlation
implied_correlation = (variance_spx - weighted_sum_variances) / (weighted_sum_products)

implied_correlation

0.5657067080044584

In [38]:
# daily returns for SPX and each ETF
returns = pd.DataFrame()
df_price2 = df_price[['SPX', 'XLU', 'XLB', 'XLRE', 'XLE', 'XLP', 'XLI', 'XLC', 'XLF', 'XLY', 'XLV', 'XLK']]


for index in df_price2.columns[1:]:
    returns[index] = df_price2[index].pct_change()

ret = pd.DataFrame()
for index in df_price2.columns[0:]:
    ret = pd.concat([ret, np.log(df_price2[index]).diff()], axis=1)


# realized_vol = returns.std() * np.sqrt(252)

# calculate the correlation marix
correlation_matrix = returns.corr()
average_realized_correlation = correlation_matrix.stack().mean()

average_realized_correlation

0.5759186743668337